In [1]:
from inference_toolbox.model import Model, add_model, delete_model
from inference_toolbox.likelihood import Likelihood
from inference_toolbox.parameter import Parameter
from visualisation_toolbox.domain import Domain
from controllers.sampler import Sampler
from visualisation_toolbox.visualiser import Visualiser
from data_processing.sim_data_processor import SimDataProcessor
from data_processing.raw_data_processor import RawDataProcessor
import numpy as np
import pandas as pd
import os

os.chdir('/PhD_project/')

/build/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sim_model = Model('log_gpm_norm').add_fixed_model_param('H', 1).add_fixed_model_param('I_y',1).add_fixed_model_param('I_z',1).add_fixed_model_param('Q',1e13)
sim_domain = Domain('cone_from_source_z_limited').add_domain_param('r',1000).add_domain_param('theta',np.pi/8).add_domain_param('source', [0,0,1]).add_domain_param('resolution',20)

sim_data_processor = SimDataProcessor('sim_data_np_0.01', sim_model, sim_domain, noise_dist='gaussian', noise_percentage=0.01)

# processor_params = {
#     'experiments_list': [
#         'Exp1',
#         'Exp2',
#         'Exp3',
#         'Exp4',
#         'Exp5',
#         'Exp6',
#         'Exp7',
#         'Exp8',
#         'Exp9',
#         'Exp10',
#         'Exp11',
#         'Exp12',
#     ],
#     'meta_data_select': 'GBR_data_summary',
#     'input_header': 'Concentration',
#     'output_header': 'Concentration',
#     'log_output_data':True,
#     'gridding': [100,100,25]
# }

# raw_data_processor = RawDataProcessor('GBR_data', 'GBR_data_gridded_100_100_25', 'GBR_processor', processor_params=processor_params)


model = Model('log_gpm_norm').add_fixed_model_param('H', 1)
likelihood = Likelihood('gaussian_percentage_error')
I_y_and_I_z = Parameter(name=['I_y','I_z'],prior_select='log_norm', multi_mode=True).add_prior_param('peak',  [[0.0725, 0.0465], [0.22, 0.2], [1.6,1.2]] ).add_prior_param('overall_scale', 0.1)
# I_y_and_I_z = Parameter(name=['I_y','I_z'],prior_select='log_norm').add_prior_param('peak', [0.35,0.26] ).add_prior_param('scale',[[1,0],[0,1]])
Q = Parameter(name='Q',prior_select='log_norm', order = 13).add_prior_param('peak',  1).add_prior_param('scale', 0.001)
# Q = Parameter(name='Q',prior_select='log_norm', order = 13).add_prior_param('peak',  3.41).add_prior_param('scale', 0.825)
# sigma = Parameter(name='sigma',prior_select='log_norm').add_prior_param('peak',  0.1).add_prior_param('scale', 1)
error = Parameter(name='error',prior_select='uniform').add_prior_param('low',  0).add_prior_param('high', 1)
# error = Parameter(name='error',prior_select='log_norm').add_prior_param('peak',  0.8).add_prior_param('scale', 0.00001)
# inference_params = pd.Series({'I_y_and_I_z': I_y_and_I_z, 'Q': Q, 'sigma':sigma})
inference_params = pd.Series({'I_y_and_I_z': I_y_and_I_z, 'Q': Q, 'error':error})

sampler = Sampler(inference_params, model, likelihood, sim_data_processor, n_samples = 10000, n_chains=3)
# sampler = Sampler(inference_params, model, likelihood, raw_data_processor, n_samples = 10000, n_chains=3)
sampler.sample_all()

visualiser = Visualiser(sampler)
visualiser.get_traceplots()

domain = Domain('cone_from_source_z_limited').add_domain_param('r',1000).add_domain_param('theta',np.pi/8).add_domain_param('source', [0,0,1]).add_domain_param('resolution',40)
domain.add_domain_param('z_slice', 0)
domain.add_domain_param('y_slice', 0)
domain.add_domain_param('x_slice', 100)

visualiser.show_predictions(domain, 'test', title = 'Concentration of Droplets', plot_type='3D')
visualiser.show_predictions(domain, 'test', title = 'Concentration of Droplets', plot_type='2D_slice')
visualiser.get_autocorrelations()
visualiser.get_summary()

Q_ref = {
            'vals' : [2.82e13, 3.11e13, 2.89e13, 4.83e13],
            'labels': ['250m','200m','750m','1000m'] 
        }

I_y_and_I_z_ref = {
            'vals' : [[0.22,0.2],[1.6,1.2],[0.11,0.08],[0.08,0.06], [0.06,0.03], [0.04,0.016]],
            'labels': ['A','B','C','D','E','F'] 
        }

visualiser.plot_prior('Q', [1e12,5e13], Q_ref)
# visualiser.plot_prior('sigma', [0.5,1])
visualiser.plot_prior('error', [0,1])
visualiser.plot_prior('I_y_and_I_z', [[0.0001,1.7],[0.0001,1.7]], I_y_and_I_z_ref)

visualiser.plot_posterior('Q', [1e12,5e13], Q_ref)
# visualiser.plot_posterior('sigma', [0.5,1])
visualiser.plot_posterior('error', [0,1])
visualiser.plot_posterior('I_y_and_I_z', [[0.0001,1.7],[0.0001,1.7]], I_y_and_I_z_ref)

Data loaded from /data/processed_sim_data/sim_data_np_0.01


sample: 100%|██████████| 15000/15000 [00:12<00:00, 1213.56it/s, 63 steps of size 2.27e-02. acc. prob=0.95] 


Samples saved to /results/inference_results/sim_data_np_0.01/general_instances/instance_1
